### Imports

In [1]:
import pandas as pd
import glob
import datetime
import os, shutil
import numpy as np

### Path set-up

In [5]:
if "DATA_DIR" not in locals():
    DATA_DIR = "./data/"
else:
    print(DATA_DIR)

if os.path.exists(DATA_DIR) and os.path.isdir(DATA_DIR):
    shutil.rmtree(DATA_DIR)
os.makedirs(os.path.dirname(DATA_DIR), exist_ok=True)

./data/


In [6]:
if "OUTPUT_DATA_FOLDER" not in locals():
    OUTPUT_DATA_FOLDER = "./output/"
else:
    print(OUTPUT_DATA_FOLDER)

if os.path.exists(OUTPUT_DATA_FOLDER) and os.path.isdir(OUTPUT_DATA_FOLDER):
    shutil.rmtree(OUTPUT_DATA_FOLDER)
os.makedirs(os.path.dirname(OUTPUT_DATA_FOLDER), exist_ok=True)

In [7]:
if "ELASTIC_INDEX" not in locals():
    ELASTIC_INDEX = "siren"
else:
    print(ELASTIC_INDEX)

### Import data

In [5]:
# Import Stock Unite Legale data
df_unite_legale = pd.read_csv(
    "https://files.data.gouv.fr/insee-sirene/StockUniteLegale_utf8.zip",
    compression="zip",
    dtype=str,
    usecols=[
        "siren",
        "dateCreationUniteLegale",
        "sigleUniteLegale",
        "prenom1UniteLegale",
        "identifiantAssociationUniteLegale",
        "trancheEffectifsUniteLegale",
        "dateDernierTraitementUniteLegale",
        "categorieEntreprise",
        "etatAdministratifUniteLegale",
        "nomUniteLegale",
        "nomUsageUniteLegale",
        "denominationUniteLegale",
        "categorieJuridiqueUniteLegale",
        "activitePrincipaleUniteLegale",
        "economieSocialeSolidaireUniteLegale",
    ],
)

In [6]:
# Rename columns
df_unite_legale = df_unite_legale.rename(
    columns={
        "dateCreationUniteLegale": "date_creation_entreprise",
        "sigleUniteLegale": "sigle",
        "prenom1UniteLegale": "prenom",
        "trancheEffectifsUniteLegale": "tranche_effectif_salarie_entreprise",
        "dateDernierTraitementUniteLegale": "date_mise_a_jour",
        "categorieEntreprise": "categorie_entreprise",
        "etatAdministratifUniteLegale": "etat_administratif_unite_legale",
        "nomUniteLegale": "nom",
        "nomUsageUniteLegale": "nom_usage",
        "denominationUniteLegale": "nom_raison_sociale",
        "categorieJuridiqueUniteLegale": "nature_juridique_entreprise",
        "activitePrincipaleUniteLegale": "activite_principale_entreprise",
        "economieSocialeSolidaireUniteLegale": "economie_sociale_solidaire_unite_legale",
        "identifiantAssociationUniteLegale": "identifiant_association_unite_legale",
    }
)

### Data preprocessing

In [7]:
def nom_complet(x):
    if x["nature_juridique_entreprise"] == "1000":
        if x["sigle"] == x["sigle"]:
            if (x["prenom"] == x["prenom"]) & (x["nom"] == x["nom"]):
                if x["nom usage"] == x["nom_usage"]:
                    return (
                        x["prenom"].lower()
                        + " "
                        + x["nom_usage"].lower()
                        + " ("
                        + x["nom"].lower()
                        + ", "
                        + x["sigle"].lower()
                        + ")"
                    )
                else:
                    return (
                        x["prenom"].lower()
                        + " "
                        + x["nom"].lower()
                        + " ("
                        + x["sigle"].lower()
                        + ")"
                    )
            else:
                return None
        else:
            if (x["prenom"] == x["prenom"]) & (x["nom"] == x["nom"]):
                if x["nom_usage"] == x["nom_usage"]:
                    return (
                        x["prenom"].lower()
                        + " "
                        + x["nom_usage"].lower()
                        + " ("
                        + x["nom"].lower()
                        + ")"
                    )
                else:
                    return x["prenom"].lower() + " " + x["nom"].lower()
            else:
                return None
    else:
        if x["sigle"] == x["sigle"]:
            if x["nom_raison_sociale"] == x["nom_raison_sociale"]:
                return x["nom_raison_sociale"].lower() + " (" + x["sigle"].lower() + ")"
            else:
                return None
        else:
            if x["nom_raison_sociale"] == x["nom_raison_sociale"]:
                return x["nom_raison_sociale"].lower()
            else:
                return None

In [8]:
# Add nom_complet column to df_unite_legale
df_unite_legale["nom_complet"] = df_unite_legale.apply(
    lambda row: nom_complet(row), axis=1
)

In [ ]:
# Create list of departement zip codes
all_deps = [
    *"-0".join(list(str(x) for x in range(0, 10))).split("-")[1:],
    *list(str(x) for x in range(10, 20)),
    *["2A", "2B"],
    *list(str(x) for x in range(21, 95)),
    *"-7510".join(list(str(x) for x in range(0, 10))).split("-")[1:],
    *"-751".join(list(str(x) for x in range(10, 21))).split("-")[1:],
    *[""],
]
# Remove Paris zip code
all_deps.remove("75")

In [10]:
# Upload geo data by departement
for dep in all_deps:
    url = "https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_" + dep + ".csv.gz"
    print(url)
    df_dep = pd.read_csv(
        url,
        compression="gzip",
        dtype=str,
        usecols=[
            "siren",
            "siret",
            "dateCreationEtablissement",
            "trancheEffectifsEtablissement",
            "activitePrincipaleRegistreMetiersEtablissement",
            "etablissementSiege",
            "numeroVoieEtablissement",
            "libelleVoieEtablissement",
            "codePostalEtablissement",
            "libelleCommuneEtablissement",
            "typeVoieEtablissement",
            "codeCommuneEtablissement",
            "complementAdresseEtablissement",
            "dateDebut",
            "etatAdministratifEtablissement",
            "enseigne1Etablissement",
            "enseigne1Etablissement",
            "enseigne2Etablissement",
            "enseigne3Etablissement",
            "denominationUsuelleEtablissement",
            "activitePrincipaleEtablissement",
            "geo_adresse",
            "longitude",
            "latitude",
            "indiceRepetitionEtablissement",
        ],
    )
    df_dep = df_dep.rename(
        columns={
            "dateCreationEtablissement": "date_creation",
            "trancheEffectifsEtablissement": "tranche_effectif_salarie",
            "activitePrincipaleRegistreMetiersEtablissement": "activite_principale_registre_metier",
            "etablissementSiege": "is_siege",
            "numeroVoieEtablissement": "numero_voie",
            "typeVoieEtablissement": "type_voie",
            "libelleVoieEtablissement": "libelle_voie",
            "codePostalEtablissement": "code_postal",
            "libelleCommuneEtablissement": "libelle_commune",
            "codeCommuneEtablissement": "commune",
            "complementAdresseEtablissement": "complement_adresse",
            "codeCedexEtablissement": "cedex",
            "dateDebut": "date_debut_activite",
            "etatAdministratifEtablissement": "etat_administratif_etablissement",
            "enseigne1Etablissement": "enseigne_1",
            "enseigne2Etablissement": "enseigne_2",
            "enseigne3Etablissement": "enseigne_3",
            "activitePrincipaleEtablissement": "activite_principale",
            "indiceRepetitionEtablissement": "indice_repetition",
            "denominationUsuelleEtablissement": "nom_commercial",
        }
    )
    df_dep.to_csv(DATA_DIR + "geo_siret_" + dep + ".csv", index=False)

https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_11.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_23.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_69.csv.gz
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_57.csv.gz


In [11]:
# Get geo data file paths
geo_files = glob.glob(DATA_DIR + "geo_siret*.csv")

In [12]:
geo_files.sort()

In [13]:
# Compute nbre d'établissements' per 'siren'
df_out = pd.DataFrame()
for geo_file in geo_files:
    print(geo_file)
    df_geo = pd.read_csv(geo_file, dtype=str)
    df_geo = df_geo.replace({np.nan: None})
    df_geo["file"] = geo_file
    # Create column with list of "enseignes" and "nom_commercial"
    df_geo["enseigne"] = df_geo.apply(
        lambda x: list(
            filter(
                None,
                set(
                    [
                        x["enseigne_1"],
                        x["enseigne_2"],
                        x["enseigne_3"],
                        x["nom_commercial"],
                    ]
                ),
            )
        ),
        axis=1,
    )
    df_geo["nombre_etablissements"] = df_geo.groupby(["siren", "file"])[
        "siret"
    ].transform("count")
    df_enseigne = (
        df_geo.groupby(["siren", "file"])["enseigne"]
        .apply(list)
        .reset_index(name="liste_enseigne_dep")
        .drop(columns=["file"], axis=1)
    )
    df_enseigne["liste_enseigne_dep"] = df_enseigne.apply(
        lambda x: list(set(c for b in x.liste_enseigne_dep for c in b)), axis=1
    )
    df_geo = df_geo.merge(df_enseigne, left_on="siren", right_on="siren")
    df_adresse = (
        df_geo.groupby(["siren", "file"])["geo_adresse"]
        .apply(set)
        .reset_index(name="liste_adresse_dep")
        .drop(columns=["file"], axis=1)
    )
    df_geo = df_geo.merge(df_adresse, left_on="siren", right_on="siren")
    df_inter = df_geo[
        [
            "siren",
            "file",
            "nombre_etablissements",
            "liste_enseigne_dep",
            "liste_adresse_dep",
        ]
    ]
    df_out = pd.concat([df_out, df_inter])

./data/geo_siret_11.csv
./data/geo_siret_23.csv
./data/geo_siret_57.csv
./data/geo_siret_69.csv


In [14]:
df_out.head(5)

siren                     file  nombre_etablissements  \
0        007171754  ./data/geo_siret_11.csv                      1   
1        007273378  ./data/geo_siret_11.csv                      1   
2        015375074  ./data/geo_siret_11.csv                      1   
3        015375736  ./data/geo_siret_11.csv                      1   
4        015376056  ./data/geo_siret_11.csv                      1   
...            ...                      ...                    ...   
1010250  999990005  ./data/geo_siret_69.csv                      5   
1010251  999990468  ./data/geo_siret_69.csv                      4   
1010252  999990468  ./data/geo_siret_69.csv                      4   
1010253  999990468  ./data/geo_siret_69.csv                      4   
1010254  999990468  ./data/geo_siret_69.csv                      4   

                    liste_enseigne_dep  \
0                               {None}   
1                               {None}   
2                               {None}   
3                               {None}   
4                               {None}   
...                                ...   
1010250  {None, STEF, STEF LOGISTIQUE}   
1010251                         {None}   
1010252                         {None}   
1010253                         {None}   
1010254                         {None}   

                                         liste_adresse_dep  
0                 {Résidence Hotel du Port 11430 Gruissan}  
1                     {45 Rue de Verdun 11000 Carcassonne}  
2                 {4 Rue des Calquières 11000 Carcassonne}  
3              {Résidence Hauts de St Pierre 11560 Fleury}  
4                            {Rue du Moulin 11370 Leucate}  
...                                                    ...  
1010250  {14 Rue Marcel Mérieux 69960 Corbas, 67 Avenue...  
1010251  {49 Rue de la République 69002 Lyon, 186 Avenu...  
1010252  {49 Rue de la République 69002 Lyon, 186 Avenu...  
1010253  {49 Rue de la République 69002 Lyon, 186 Avenu...  
1010254  {49 Rue de la République 69002 Lyon, 186 Avenu...  

[1646089 rows x 5 columns]

In [15]:
df_out = df_out.drop_duplicates(subset=["siren", "file"], keep="first")
df_liste = (
    df_out.groupby(["siren"])["liste_enseigne_dep"]
    .apply(list)
    .reset_index(name="liste_enseigne")
)
df_out = df_out.merge(df_liste, left_on="siren", right_on="siren")
df_liste = (
    df_out.groupby(["siren"])["liste_adresse_dep"]
    .apply(list)
    .reset_index(name="liste_adresse")
)
df_out = df_out.merge(df_liste, left_on="siren", right_on="siren")

In [16]:
df_out2 = (
    df_out[["siren", "nombre_etablissements"]].groupby(["siren"], as_index=False).sum()
)

In [17]:
df_out2 = df_out2.merge(
    df_out[["liste_enseigne", "liste_adresse", "siren"]], on="siren", how="left"
)
df_out2 = df_out2.drop_duplicates(subset=["siren"], keep="first")

In [18]:
df_out2["liste_enseigne"] = df_out2.apply(
    lambda x: list(set(c for b in x.liste_enseigne for c in b)), axis=1
)
df_out2["liste_adresse"] = df_out2.apply(
    lambda x: list(set(c for b in x.liste_adresse for c in b)), axis=1
)

In [19]:
df_out2

siren  nombre_etablissements  \
0        005720651                      1   
1        006350037                      1   
2        006580195                      1   
3        006970859                      1   
4        007080773                      1   
...            ...                    ...   
1251979  998912307                      1   
1251980  998922108                      4   
1251981  999990005                      5   
1251982  999990468                      4   
1251983  999990625                      1   

                               liste_enseigne  \
0                                      [None]   
1                                      [None]   
2                                     [SIDES]   
3                                      [None]   
4                               [DOLMEN LYON]   
...                                       ...   
1251979                                [None]   
1251980  [None, SAN MARINA, CHAUSSURES LAITI]   
1251981         [None, STEF, STEF LOGISTIQUE]   
1251982                                [None]   
1251983                                [None]   

                                             liste_adresse  
0                     [10 Rue Général Plessier 69002 Lyon]  
1                           [66 Rue Béchevelin 69007 Lyon]  
2                   [7 Chemin de Genas 69800 Saint-Priest]  
3                             [54 Cours Vitton 69006 Lyon]  
4                     [Route de Rive-de-Gier 69700 Givors]  
...                                                    ...  
1251979                   [30 Rue de Lorquin 57400 Imling]  
1251980  [Voie Romaine 57280 Semécourt, Rue du Maillet ...  
1251981  [14 Rue Marcel Mérieux 69960 Corbas, 67 Avenue...  
1251982  [49 Rue de la République 69002 Lyon, 11 Rue de...  
1251983                  [Route de Manom 57100 Thionville]  

[1246465 rows x 4 columns]

In [20]:
df_unite_legale = pd.merge(df_unite_legale, df_out2, on="siren", how="left")

In [21]:
df_unite_legale

siren date_creation_entreprise sigle          prenom  \
0         000325175               2000-09-26   NaN         THIERRY   
1         001807254               1972-05-01   NaN  JACQUES-LUCIEN   
2         005410220               1954-12-25   NaN         GEORGES   
3         005410345                      NaN   NaN          MICHEL   
4         005410394               1954-12-25   NaN          ROBERT   
...             ...                      ...   ...             ...   
22874622  999990625               1983-11-01   NaN             NaN   
22874623  999990641               1983-11-01   NaN             NaN   
22874624  999990666               1986-05-15   NaN             NaN   
22874625  999990682               1985-09-01   NaN             NaN   
22874626  999992357               2003-12-31   NaN             NaN   

         identifiantAssociationUniteLegale  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   
...                                    ...   
22874622                               NaN   
22874623                               NaN   
22874624                               NaN   
22874625                               NaN   
22874626                               NaN   

         tranche_effectif_salarie_entreprise     date_mise_a_jour  \
0                                        NaN  2019-12-13T13:21:28   
1                                        NaN  2016-07-10T05:00:06   
2                                        NaN                  NaN   
3                                        NaN                  NaN   
4                                        NaN                  NaN   
...                                      ...                  ...   
22874622                                 NaN                  NaN   
22874623                                 NaN                  NaN   
22874624                                  NN  2021-09-17T08:46:57   
22874625                                 NaN                  NaN   
22874626                                  01  2021-10-27T10:03:28   

         categorie_entreprise etatAdministratifUniteLegale        nom  \
0                         PME                            A    JANOYER   
1                         NaN                            C     BRETON   
2                         NaN                            C  WATTEBLED   
3                         NaN                            C     DEBRAY   
4                         NaN                            C      DAULT   
...                       ...                          ...        ...   
22874622                  NaN                            C        NaN   
22874623                  NaN                            C        NaN   
22874624                   GE                            A        NaN   
22874625                  NaN                            C        NaN   
22874626                  PME                            A        NaN   

                               nom_raison_sociale nature_juridique_entreprise  \
0                                             NaN                        1000   
1                                             NaN                        1000   
2                                             NaN                        1000   
3                                             NaN                        1000   
4                                             NaN                        1000   
...                                           ...                         ...   
22874622  SOC NOUVELLE LAMINOIRS FROID THIONVILLE                        5599   
22874623               SOC DE CARBURANT MECANIQUE                        5499   
22874624                                 ASSU VIE                        5599   
22874625                                  COFICEM                        5599   
22874626               

In [22]:
# Compute 'nombre etablissements ouvert' per 'siren'
df_out = pd.DataFrame()
for geo_file in geo_files:
    print(geo_file)
    df_geo = pd.read_csv(geo_file, dtype=str)
    df_geo = df_geo[df_geo["etat_administratif_etablissement"] == "A"]
    df_geo["file"] = geo_file
    df_geo["nombre_etablissements_ouvert"] = df_geo.groupby(["siren", "file"])[
        "siret"
    ].transform("count")
    df_inter = df_geo[["siren", "file", "nombre_etablissements_ouvert"]]
    df_out = pd.concat([df_out, df_inter])
df_out = df_out.drop_duplicates(keep="first")
df_out2 = (
    df_out[["siren", "nombre_etablissements_ouvert"]]
    .groupby(["siren"], as_index=False)
    .sum()
)
df_unite_legale = pd.merge(df_unite_legale, df_out2, on="siren", how="left")

./data/geo_siret_11.csv
./data/geo_siret_23.csv
./data/geo_siret_57.csv
./data/geo_siret_69.csv


In [23]:
# Merge geo files with above dataframe
for geo_file in geo_files:
    print(geo_file)
    df_geo = pd.read_csv(geo_file, dtype=str)
    df_inter = pd.merge(df_geo, df_unite_legale, on="siren", how="left")
    df_inter2 = df_inter[df_inter["is_siege"] == "true"]
    df_inter2["concat_nom_adr_siren"] = (
        df_inter2["nom_complet"]
        + " "
        + df_inter2["geo_adresse"]
        + " "
        + df_inter2["siren"]
    )
    # df_inter2['concat_enseigne_adresse'] = df_inter2.apply(lambda x: set.union(x.liste_enseigne, x.liste_adresse), axis=1)
    df_inter2["concat_enseigne_adresse"] = (
        df_inter2["liste_enseigne"] + df_inter2["liste_adresse"]
    )
    df_inter.to_csv(
        OUTPUT_DATA_FOLDER + "siret_" + geo_file.replace(DATA_DIR + "geo_siret_", ""),
        index=False,
    )
    df_inter2.to_csv(
        OUTPUT_DATA_FOLDER
        + ELASTIC_INDEX
        + "_"
        + geo_file.replace(DATA_DIR + "geo_siret_", ""),
        index=False,
    )

./data/geo_siret_11.csv


/tmp/ipykernel_213/1469716798.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inter2['concat_nom_adr_siren'] = df_inter2['nom_complet'] + ' ' + df_inter2['geo_adresse'] + ' ' + df_inter2['siren']
/tmp/ipykernel_213/1469716798.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inter2['concat_enseigne_adresse'] = df_inter2['liste_enseigne'] + df_inter2['liste_adresse']


./data/geo_siret_23.csv


/tmp/ipykernel_213/1469716798.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inter2['concat_nom_adr_siren'] = df_inter2['nom_complet'] + ' ' + df_inter2['geo_adresse'] + ' ' + df_inter2['siren']
/tmp/ipykernel_213/1469716798.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inter2['concat_enseigne_adresse'] = df_inter2['liste_enseigne'] + df_inter2['liste_adresse']


./data/geo_siret_57.csv


/tmp/ipykernel_213/1469716798.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inter2['concat_nom_adr_siren'] = df_inter2['nom_complet'] + ' ' + df_inter2['geo_adresse'] + ' ' + df_inter2['siren']
/tmp/ipykernel_213/1469716798.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inter2['concat_enseigne_adresse'] = df_inter2['liste_enseigne'] + df_inter2['liste_adresse']


./data/geo_siret_69.csv


/tmp/ipykernel_213/1469716798.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inter2['concat_nom_adr_siren'] = df_inter2['nom_complet'] + ' ' + df_inter2['geo_adresse'] + ' ' + df_inter2['siren']
/tmp/ipykernel_213/1469716798.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inter2['concat_enseigne_adresse'] = df_inter2['liste_enseigne'] + df_inter2['liste_adresse']


In [25]:
df_inter2.head(5)

siren           siret date_creation tranche_effectif_salarie  \
0   005720651  00572065100040    2014-01-01                      NaN   
3   006970859  00697085900044    1985-09-16                       NN   
5   015450232  01545023200044    2016-09-30                      NaN   
17  015750797  01575079700043    1980-11-01                      NaN   
19  015752074  01575207400086    1997-10-01                      NaN   

   activite_principale_registre_metier is_siege numero_voie indice_repetition  \
0                                  NaN     true          10               NaN   
3                                  NaN     true          54               NaN   
5                                  NaN     true          39               NaN   
17                                 NaN     true          13               NaN   
19                                 NaN     true         NaN               NaN   

        libelle_voie code_postal  ... nature_juridique_entreprise  \
0   GENERAL PLESSIER       69002  ...                        5499   
3             VITTON       69006  ...                        1000   
5             AMPERE       69680  ...                        5499   
17       FERRANDIERE       69002  ...                        5499   
19     VALLEE D'OZON       69970  ...                        5599   

   activite_principale_entreprise economieSocialeSolidaireUniteLegale  \
0                          68.20B                                   N   
3                           52.5Z                                 NaN   
5                          70.22Z                                   N   
17                          52.1J                                 NaN   
19                          51.5L                                 NaN   

                        nom_complet nombre_etablissements liste_enseigne  \
0            sa des ets rene dufour                   1.0         [None]   
3                     serge lacombe                   1.0         [None]   
5                               t21                   1.0         [None]   
17            etablissements comexi                   2.0         [None]   
19  etablissements desgrange freres                   1.0         [None]   

                                        liste_adresse  \
0                [10 Rue Général Plessier 69002 Lyon]   
3                        [54 Cours Vitton 69006 Lyon]   
5                      [39 Rue Ampère 69680 Chassieu]   
17  [Rue Ferrandière 69002 Lyon, 6 Cours Lafayette...   
19                 [Chemin de l’Ozon 69970 Chaponnay]   

   nombre_etablissements_ouvert  \
0                           1.0   
3                           NaN   
5                           1.0   
17                          NaN   
19                          NaN   

                                 concat_nom_adr_siren  \
0   sa des ets rene dufour 10 Rue Général Plessier...   
3   serge lacombe 54 Cours Vitton 69006 Lyon 00697...   
5          t21 39 Rue Ampère 69680 Chassieu 015450232   
17  etablissements comexi Rue Ferrandière 69002 Ly...   
19  etablissements desgrange freres Chemin de l’Oz...   

                              concat_enseigne_adresse  
0          [None, 10 Rue Général Plessier 69002 Lyon]  
3                  [None, 54 Cours Vitton 69006 Lyon]  
5                [None, 39 Rue Ampère 69680 Chassieu]  
17  [None, Rue Ferrandière 69002 Lyon, 6 Cours Laf...  
19           [None, Chemin de l’Ozon 69970 Chaponnay]  

[5 rows x 39 columns]